In [1]:
import pandas as pd
import geopandas as gpd

In [8]:
# importar capas
gdf_evp = gpd.read_file("./CapasEditadas/EspaciosPublicos_v10.geojson")
gdf_evp.drop(columns=['sis_area_m2', 'barpop_nam', 'barpop_cant', 'bus50_cant', 'bus500_cant'], inplace=True)

In [39]:
# importar capas de municipios y límite antrópico
gdf_muni = gpd.read_file("../1_Datos/ACUMAR/LIMITE_POLITICO.shp")
gdf_limite = gpd.read_file("../1_Datos/ACUMAR/LIMITE_ANTROPICO_POLIGONO.geojson")

In [5]:
# importar capas de línea de colectivos
gdf_bus = gpd.read_file("./Capas_Output/LineasColectivos_v1.geojson")

In [6]:
# importar barrios populares
gdf_barrios = gpd.read_file("../1_Datos/Renabap/BarriosPopulares_2022-05-15_info_publica.geojson")
gdf_barrios.head()

,renabap_id,nombre_barrio,provincia,departamento,localidad,cantidad_familias_aproximada,cantidad_viviendas_aproximadas,decada_de_creacion,anio_de_creacion,energia_electrica,efluentes_cloacales,agua_corriente,cocina,calefaccion,situacion_dominial,clasificacion_barrio,superficie_m2,geometry
0,1,Monterrey I,Buenos Aires,Pilar,Presidente Derqui,44.0,40.0,Década 1990,NaN,Conexión a la red con medidor comunitario,Desagüe a cámara séptica y pozo ciego,Bomba de agua de pozo domiciliaria,Gas en garrafa,Otro / vacío,Otro tipo de seguridad en la tenencia,Asentamiento,11674,"MULTIPOLYGON (((-58.83350 -34.48128, -58.83383..."
1,2,Malvinas II,Buenos Aires,La Plata,José Melchor Romero,319.0,290.0,Década 1990,NaN,Conexión irregular a la red,Desagüe a cámara séptica y pozo ciego,Conexión irregular a la red de agua,Gas en garrafa,Otro / vacío,Ninguna seguridad en la tenencia,Asentamiento,98034,"MULTIPOLYGON (((-58.01200 -34.94732, -58.00911..."
2,3,Ferroviario,Buenos Aires,La Plata,Angel Etcheverry,146.0,133.0,Década 2000,NaN,Conexión irregular a la red,Desagüe sólo a pozo negro/ciego u hoyo,Conexión formal a la red de agua con factura,Gas en garrafa,Leña o carbón,Ninguna seguridad en la tenencia,Asentamiento,75850,"MULTIPOLYGON (((-58.07980 -35.03507, -58.07958..."
3,4,La Favelita,Buenos Aires,La Plata,Tolosa,134.0,122.0,Década 1970,NaN,Conexión irregular a la red,Desagüe sólo a pozo negro/ciego u hoyo,Conexión irregular a la red de agua,Gas en garrafa,Energía eléctrica,Ninguna seguridad en la tenencia,Asentamiento,36865,"MULTIPOLYGON (((-57.97991 -34.90834, -57.97994..."
4,5,Casaca,Buenos Aires,La Plata,City Bell,22.0,20.0,Década 2000,NaN,Conexión irregular a la red,Desagüe a cámara séptica y pozo ciego,Bomba de agua de pozo domiciliaria,Gas en garrafa,Otro / vacío,Ninguna seguridad en la tenencia,Asentamiento,25754,"MULTIPOLYGON (((-58.06297 -34.90134, -58.06328..."


# Crear nuevo campo bool y nombre de barrios popular

In [ ]:
# presencia y nombre de barrio al que pertenece el espacio verde
""" gdf_evp_b = gpd.sjoin(gdf_evp, gdf_barrios, how='left')
gdf_evp_b.drop_duplicates(subset=['full_id'], inplace=True)
gdf_evp = gdf_evp.merge(gdf_evp_b[['full_id', 'nombre_barrio']], on='full_id', how='left')
gdf_evp.rename(columns={'nombre_barrio':'barp_nam'}, inplace=True)
gdf_evp['barp_bool'] = 0
gdf_evp.loc[gdf_evp.barp_nam.notnull(), 'barp_bool'] = 1 """

In [10]:
# buffer de 200m y cruce de capas
gdf_evp_buffer = gdf_evp.copy()
gdf_evp_buffer = gdf_evp_buffer.to_crs("EPSG:5347")
gdf_evp_buffer.geometry = gdf_evp_buffer.buffer(200)
gdf_evp_buffer = gdf_evp_buffer.to_crs("EPSG:4326")

gdf_evp_buffer_b = gpd.sjoin(gdf_evp_buffer, gdf_barrios, how='left')

In [11]:
# cantidad y nombres de barrios a 200 de cada espacio verde
pd_bar100_size = pd.DataFrame(gdf_evp_buffer_b.loc[gdf_evp_buffer_b.nombre_barrio.notnull(), 'full_id'].value_counts())
pd_bar100_size.reset_index(inplace=True)
pd_bar100_size.rename(columns={'index':'full_id', 'full_id':'barpop_cant'}, inplace=True)
pd_bar100_nam = pd.DataFrame(gdf_evp_buffer_b.loc[gdf_evp_buffer_b.nombre_barrio.notnull()].groupby(['full_id'])['nombre_barrio'].apply(lambda x: (", ").join(list(set(x)))))
pd_bar100_nam.reset_index(inplace=True)

In [12]:
gdf_evp = gdf_evp.merge(pd_bar100_size[['full_id', 'barpop_cant']], on='full_id', how='left')
gdf_evp = gdf_evp.merge(pd_bar100_nam[['full_id', 'nombre_barrio']], on='full_id', how='left')
gdf_evp.rename(columns={'nombre_barrio':'barpop_nam'}, inplace=True)
gdf_evp.barpop_cant.fillna(0, inplace=True)

# Generar valores de campo "full_id" para los datos UNSAM 

In [ ]:
""" num = 1
for i in gdf_evp.loc[gdf_evp.fuente == "UNSAM"].index:
    num_str = str(num).zfill(5)
    gdf_evp.loc[i, 'full_id'] = "usm" + num_str
    num = num+1 """

# Suma de área de sistemas líneales

In [13]:
gdf_evp.loc[gdf_evp.sis_nam == '', 'sis_nam'] = None

In [14]:
df_sissup = pd.DataFrame(gdf_evp.loc[(gdf_evp.sis_nam.notnull())&(gdf_evp.sis_nam != '')].groupby("sis_nam")['area_m2'].sum()).reset_index()
df_sissup.rename(columns={'area_m2':'sis_area_m2'}, inplace=True)

In [15]:
print(len(gdf_evp))
gdf_evp = gdf_evp.merge(df_sissup, on='sis_nam', how='left')
print(len(gdf_evp))

302
302


# Cruce con líneas de colectivos

In [16]:
# buffer de 50m y cruce capa capa de líenas de colectivos
gdf_evp_buffer_50m = gdf_evp.copy()
gdf_evp_buffer_50m = gdf_evp_buffer_50m.to_crs("EPSG:5347")
gdf_evp_buffer_50m.geometry = gdf_evp_buffer_50m.buffer(50)
gdf_evp_buffer_50m = gdf_evp_buffer_50m.to_crs("EPSG:4326")

gdf_evp_buffer_50m_sj = gpd.sjoin(gdf_evp_buffer_50m, gdf_bus, how='left')
df_bus50 = pd.DataFrame(gdf_evp_buffer_50m_sj.loc[gdf_evp_buffer_50m_sj.full_id_right.notnull(), 'full_id_left'].value_counts()).reset_index().rename(columns={'index':'full_id', 'full_id_left':'bus50_cant'})

In [17]:
# buffer de 500m y cruce capa capa de líenas de colectivos
gdf_evp_buffer_500m = gdf_evp.copy()
gdf_evp_buffer_500m = gdf_evp_buffer_500m.to_crs("EPSG:5347")
gdf_evp_buffer_500m.geometry = gdf_evp_buffer_500m.buffer(500)
gdf_evp_buffer_500m = gdf_evp_buffer_500m.to_crs("EPSG:4326")

gdf_evp_buffer_500m_sj = gpd.sjoin(gdf_evp_buffer_500m, gdf_bus, how='left')
df_bus500 = pd.DataFrame(gdf_evp_buffer_500m_sj.loc[gdf_evp_buffer_500m_sj.full_id_right.notnull(), 'full_id_left'].value_counts()).reset_index().rename(columns={'index':'full_id', 'full_id_left':'bus500_cant'})

In [18]:
print(len(gdf_evp))
gdf_evp = gdf_evp.merge(df_bus50, on='full_id', how='left')
gdf_evp = gdf_evp.merge(df_bus500, on='full_id', how='left')
gdf_evp.bus50_cant.fillna(0, inplace=True)
gdf_evp.bus500_cant.fillna(0, inplace=True)
print(len(gdf_evp))

302
302


# Limpieza de datos

In [19]:
# check unique ids
len(gdf_evp) == len(gdf_evp.full_id.unique())

True

In [20]:
# check unique tipo
gdf_evp.tipo.unique()

array(['Plaza', 'Plazoleta', 'Parque con equipamiento', '-', 'Parque',
       'Separador vial', 'Cancha'], dtype=object)

In [25]:
print(gdf_evp.loc[gdf_evp.tipo == '-', 'evp'].unique())
print(gdf_evp.loc[gdf_evp.tipo == 'Cancha', 'evp'].unique())
print(gdf_evp.loc[gdf_evp.tipo == 'Separador vial', 'evp'].unique())
print(gdf_evp.loc[gdf_evp.evp == 0, 'tipo'].unique())

[0.]
[1.]
[0.]
['-' 'Separador vial']


In [26]:
# check unique distrito
gdf_evp.distrito.unique()

array(['Lomas de Zamora', 'Lanús'], dtype=object)

In [27]:
gdf_evp[['full_id', 'fuente', 'distrito', 'evp', 'tipo', 'nombre', 'limite', 'area_m2', 
       'sis_bool', 'sis_nam', 'sis_area_m2', 'barpop_cant', 'barpop_nam', 'bus50_cant', 'bus500_cant', 
       'observaciones', 'geometry']].to_file("./CapasEditadas/EspaciosPublicos_v10.1.geojson", driver="GeoJSON")

# Capas para mapas

In [30]:
gdf_evp[['full_id', 'fuente', 'distrito', 'evp', 'tipo', 'nombre', 'limite', 'area_m2', 
       'sis_bool', 'sis_nam', 'sis_area_m2', 'barpop_cant', 'barpop_nam', 'bus50_cant', 'bus500_cant', 
       'observaciones', 'geometry']].to_file("./Mapas/MapLayouts/shp/EspaciosPublicos.shp")

c:\Users\Compumar\anaconda3\envs\GeoAnalysis\lib\site-packages\ipykernel_launcher.py:3: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
gdf_limite = gdf_limite.dissolve('Layer')
gdf_limite = gdf_limite[['geometry']] 
gdf_limite.to_file("./Mapas/MapLayouts/shp/LimiteAntropico.shp")

In [44]:
gdf_muni_inersect = gdf_muni.overlay(gdf_limite, how='intersection')
gdf_muni_inersect.to_file("./Mapas/MapLayouts/shp/LimitePolíticoAntropico.shp")

In [46]:
gdf_muni.to_file("./Mapas/MapLayouts/shp/LimitePolítico.shp")